In [1]:
from ddf_utils.factory import ihme

In [2]:
md = ihme.load_metadata()

In [3]:
md.keys()

dict_keys(['age', 'cause', 'groups', 'location', 'measure', 'metric', 'rei', 'sex', 'year', 'year_range', 'version'])

In [4]:
md['version'].sort_values(by='version_id').iloc[-1]

version_id                                                   218
description    CodCorrect v66, \r\nMMR v66, Burdenator v27, D...
metrics        [4, 2, 5, 22, 6, 23, 24, 25, 26, 28, 29, 21, 3...
date                                            09/18/2017 15:34
status                                                         1
sort                                                          40
Name: 218, dtype: object

In [9]:
ihme.bulk_download('./source/', 218, 'le')

working on https://s3.healthdata.org/gbd-api-2016-production/b55a567528401b9aaf771526c6c9ba63


True

In [29]:
import os

In [ ]:
os.remove('./source/IHME-GBD_2016_DATA-b55a5675-1.zip')

In [6]:
import zipfile

In [10]:
z = zipfile.ZipFile('./source/IHME-GBD_2016_DATA-b55a5675-1.zip')

In [15]:
z.namelist()

['IHME-GBD_2016_DATA-b55a5675-1.csv', 'citation.txt']

In [18]:
from io import BytesIO

In [11]:
import pandas as pd

In [23]:
f = BytesIO(z.read('IHME-GBD_2016_DATA-b55a5675-1.csv'))

In [24]:
data = pd.read_csv(f)

In [25]:
data.head()

,measure,location,sex,age,metric,year,val,upper,lower
0,26,44634,1,5,5,2009,70.791330,71.226869,70.325250
1,26,44634,2,5,5,2009,77.717584,78.044549,77.397962
2,26,44634,3,5,5,2009,74.206395,74.481416,73.908260
3,26,44634,1,6,5,2009,67.000202,67.427518,66.530097
4,26,44634,2,6,5,2009,73.912079,74.239210,73.589771


In [26]:
data.measure.unique()

array([26])

In [27]:
data.metric.unique()

array([5])

In [30]:
data = data.drop(['measure', 'metric'], axis=1)

In [32]:
data = data.set_index(['location', 'sex', 'age', 'year'])

In [33]:
data = data.rename(columns={'val': 'mean'})

In [36]:
for c in data.columns:
    data[[c]].to_csv('../ddf--datapoints--{}--by--location--sex--age--year.csv'.format(c))

In [50]:
concepts = {}

concepts['measure'] = data.columns.values

In [51]:
concepts['entity_domain'] = ['location', 'sex', 'age']

In [21]:
md['location'].head()

,location_id,type,name,medium_name,short_name,enabled
1,1,NaN,Global,Global,Earth,NaN
10,10,NaN,Cambodia,Cambodia,Cambodia,NaN
100,100,NaN,High-income North America,High-income North America,HI N Am,NaN
101,101,NaN,Canada,Canada,Canada,NaN
102,102,NaN,United States,United States,USA,NaN


In [39]:
str_concepts = set()

In [40]:
for e in ['location', 'sex', 'age']:
    edf = md[e]
    edf = edf.rename(columns={e+'_id': e})
    edf = edf.set_index(e)
    for c in edf.columns:
        str_concepts.add(c)
    edf.to_csv('../ddf--entities--{}.csv'.format(e))

In [41]:
str_concepts

{'enabled',
 'label',
 'medium_name',
 'name',
 'plot',
 'short_name',
 'sort',
 'type'}

In [52]:
concepts['string'] = list(str_concepts)
concepts['time'] = ['year']

In [57]:
clist = []

for k, v in concepts.items():
    for v_ in v:
        clist.append({'concept': v_, 'concept_type': k})

In [59]:
cdf = pd.DataFrame.from_records(clist)

In [60]:
cdf

,concept,concept_type
0,mean,measure
1,upper,measure
2,lower,measure
3,location,entity_domain
4,sex,entity_domain
5,age,entity_domain
6,plot,string
7,medium_name,string
8,name,string
9,type,string


In [61]:
cdf.to_csv('../ddf--concepts.csv', index=False)